In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
import pandas as pd


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/LaptrinhAI/hanoi-aqi-weather-data.csv',
                 parse_dates=['Local Time', 'UTC Time'],
                 index_col=['Local Time'])
df.head()

In [ ]:
df.hist(bins=50, figsize=(15,15));

# Data Wrangle


## Split data train, test

In [ ]:
# the idea is the split the dataset into two parts, one for training, and one for validation
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data)*test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
# now we can split them with a ratio like this
train_set, test_set = split_train_test(df, 0.2)

In [ ]:
len(train_set)

In [ ]:
len(test_set)

In [ ]:
# however, the sklearn library has such unility
from sklearn.model_selection import train_test_split

In [ ]:
train_set, test_set = train_test_split(df, test_size=0.2, random_state=2020)
print(len(train_set))
train_set.head()

## Fill NaN values

In [ ]:
# make a copy and test out
df1 = df.copy(deep=True)

In [ ]:
# calculate the median of the Windspeed (WS) input
median = df1['Wind Speed'].median()

In [ ]:
df1.head(5)

In [ ]:
df1['Wind Speed'].fillna(median, inplace=True)

In [ ]:
df1.info()

In [ ]:
# and sklearn has the Class to do such
from sklearn.impute import SimpleImputer


In [ ]:
inputer = SimpleImputer(strategy='median')

In [ ]:
df2 =df[df.columns[5:]].copy(deep=True)
df2.info()

In [ ]:
# evaluate df2
inputer.fit(df2)

In [ ]:
# see the statistic, median in this case
inputer.statistics_

In [ ]:
df2.head(5)

In [ ]:
# transform is doing the work
df2full = inputer.transform(df2)

In [ ]:
# convert the inputed dataset and  compared
df2 = pd.DataFrame(data=df2full, columns=df1.columns[5:])
df2.info()

In [ ]:
# you can save to a file with all missing values filled
df.to_csv('/content/drive/MyDrive/LaptrinhAI/filled_PM2.5_Hanoi_2018.csv')

## Additional Cleanup

In [ ]:
cot = df.columns
cot

In [ ]:
# let return back to the original dataset (df) before fill up NaN
df_num = df[['AQI', 'CO', 'NO2',
       'O3', 'PM10', 'PM25', 'SO2', 'Clouds', 'Precipitation', 'Pressure',
       'Relative Humidity', 'Temperature', 'UV Index', 'Wind Speed']]
data = df_num.copy(deep=True)
sns.heatmap(data.corr(), cmap='seismic')

In [ ]:
data.head(1)

In [ ]:
# and only correlation with PM2.5
data.corr()['AQI'].sort_values().to_frame().drop('AQI').plot.barh()

In [ ]:
data.columns

In [ ]:
# drop some columns either weak in correation or dependent (redundant) to other inputs
data.drop(columns=['CO', 'Clouds', 'UV Index', 'SO2'], inplace=True)

In [ ]:
data.head()

In [ ]:
data.info()

Split features(meteorological inputs) and label (AQI)

In [ ]:
# let make X as the matrix for the feature (or inputs)
X = data.drop('AQI', axis=1)

In [ ]:
# and lowercase y as the label (or the value of the target)
y = data['AQI'].copy()

In [ ]:
# let build the inpute instance to work with whole data at one
# to inpute more than one columns, we can use this
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:

# transform data from real value to a relative to the set
from sklearn.preprocessing import StandardScaler

In [ ]:

# we have all column with numeric values
num_attrs = list(data.columns)
num_attrs.remove('AQI')
num_attrs

In [ ]:
# first is the trategy for inputer using median
# then convert the absolute value in the each column using the Standard Class
num_pipeline = Pipeline([
        ('inputer', SimpleImputer(strategy='median')),
        ('std_scaler', StandardScaler()),
        ])

In [ ]:
# and instance to tranform all column at one
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attrs)
])

In [ ]:
# or building a function to do all in one step
def inpute_transfrom(data=None):
    num_pipeline = Pipeline([
        ('inputer', SimpleImputer(strategy='median')),
        ('std_scaler', StandardScaler()),
        ])
    num_attrs = list(data.columns)
    full_pipeline = ColumnTransformer([
        ('num', num_pipeline, num_attrs)
        ])
    return full_pipeline.fit_transform(data) # return a numpy array


In [ ]:
X_scaled = inpute_transfrom(data=X)

In [ ]:
# how do we know that the data has been fixed properly
X_scaled_test = inpute_transfrom(data=X)
dft = pd.DataFrame(data=X_scaled_test, columns=num_attrs)
dft.info()
# looking good
del dft

In [ ]:
# now we can split data, the test_size is the portion of data use for validation
# random_state is to provide consistent if you want to replicate the result
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y, test_size=0.33, random_state=2020)

# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lin_reg = LinearRegression()

In [ ]:
# and training the model using the _train dataset
lin_reg.fit(X_train, y_train)

In [ ]:
lin_reg.get_params()

In [ ]:
# let see the output of the mode
lin_reg.coef_

In [ ]:
# predict based on the training set
y_train_prd = lin_reg.predict(X_train)

In [ ]:
# let see how the label (y_train), and predicting of the label (y_prd)
plt.figure(figsize=(15,5))
plt.plot(y_train.to_list())#mau xanh
plt.plot(y_train_prd)#mau vang
plt.xlim(0,500)

Nhận xét: Model chưa dự đoán được


In [ ]:
# more important, how about with validation set (test set)
y_test_prd = lin_reg.predict(X_test)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(y_test.to_list())
plt.plot(y_test_prd)
plt.xlim(0,500)

Nhận xét: model dự đoán chưa chính xác

# evaluate model performance

In [ ]:
# for numeric data, one simple way to to see how far
# between the prediction and the garget
from sklearn.metrics import mean_squared_error

In [ ]:
# on training set
lin_train_mse = mean_squared_error(y_train, y_train_prd)
print('Trainset: Root Squared Mean Error', np.sqrt(lin_train_mse))

Nhận xét: RMSE(độ lệch bình phương giữa giá trị thực tế và giá trị dụ đoán)

In [ ]:
# on test set
lin_test_mse = mean_squared_error(y_test, y_test_prd)
print('Test set: Root Squared Mean Error', np.sqrt(lin_test_mse))

Nhận xét: RMSE thấp hơn trên testset, cho thấy model chưa hoàn toán bị overfitting, nhưng có khả năng khái quát tốt hơn cho dữ liệu mới

In [ ]:
# the average value label set (y set)
y.mean()

In [ ]:
# relative error
print(f'Relative Error: {100*np.sqrt(lin_test_mse)/y.mean():.0f}%')

In [ ]:
results = dict()
def add_stats(model=None, train_rmse=None, test_rmse=None):
    global results
    results[model] = {'train_rmse': round(train_rmse,1),
                     'test_rmse': round(test_rmse, 1)}
    return None

In [ ]:
add_stats(model='linear reg',
         train_rmse=np.sqrt(lin_train_mse),
         test_rmse=np.sqrt(lin_test_mse))
results

# DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeRegressor



In [ ]:
tree_reg = DecisionTreeRegressor()

In [ ]:
tree_reg.fit(X_train, y_train)

In [ ]:
tree_reg.feature_importances_

In [ ]:
tree_ytrain_d = tree_reg.predict(X_train)

In [ ]:
# no error, too good!
tree_train_rmse = np.sqrt(mean_squared_error(y_train, tree_ytrain_d))
tree_train_rmse

In [ ]:
tree_ytest_d = tree_reg.predict(X_test)

In [ ]:
tree_test_rmse = np.sqrt(mean_squared_error(y_test, tree_ytest_d))
tree_test_rmse # higher than regression,

In [ ]:
# let bag the result
add_stats(model='decisiontree reg',
         train_rmse=tree_train_rmse,
         test_rmse=tree_test_rmse)
results

In [ ]:
# the results by train set and test set are rather different, to see it
def plot_prediction(label=None, prediction=None, limit=200):
    plt.figure(figsize=(14,6))
    plt.plot(label.to_list())
    plt.plot(prediction, 'ro')
    plt.xlim(0, limit)
    return None

In [ ]:
plot_prediction(y_train, tree_ytrain_d)

In [ ]:
plot_prediction(y_test, tree_ytest_d)

Overfitting trên cả trainset và testset

# RandomForest

In [ ]:
# more powerful model
from sklearn.ensemble import RandomForestRegressor

In [ ]:
forest_reg = RandomForestRegressor()
forest_reg.fit(X_train, y_train)

In [ ]:
forest_ytrain_p = forest_reg.predict(X_train)

In [ ]:

mse_train = mean_squared_error(y_train, forest_ytrain_p)
rmse_train = np.sqrt(mse_train)
rmse_train

In [ ]:
# test set
forest_ytest_p = forest_reg.predict(X_test)

In [ ]:
mse_test = mean_squared_error(y_test, forest_ytest_p)
rmse_test = np.sqrt(mse_test)
rmse_test

In [ ]:
add_stats(model='randomforest reg',
         train_rmse=rmse_train,
         test_rmse=rmse_test)
results

Nhận xét: Linear regression không phù hợp

# Cross validation

In [ ]:
from sklearn.model_selection import cross_val_score


In [ ]:
scores = cross_val_score(tree_reg, X_train, y_train,
                        scoring='neg_mean_squared_error', cv=10)

In [ ]:
tree_rmse_scores = np.sqrt(-scores)
tree_rmse_scores

In [ ]:
def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("Standard Deviation: ", scores.std())

In [ ]:
display_scores(scores)

In [ ]:
lin_scores = cross_val_score(lin_reg, X_train, y_train,
                             scoring='neg_mean_squared_error', cv=10)


In [ ]:
display_scores(lin_scores)

In [ ]:
# how about on test set:
for model in [lin_reg, tree_reg, forest_reg]:
    scores = cross_val_score(model, X_test, y_test,
                             scoring='neg_mean_squared_error', cv=10)
    display_scores(scores)
    print('-'*40)


# Save model

In [ ]:
!pip install joblib

In [ ]:
# just in case you want to save your work
import joblib

In [ ]:
import os
os.makedirs('/content/drive/MyDrive/LaptrinhAI/model')

In [ ]:
joblib.dump(forest_reg, '/content/drive/MyDrive/LaptrinhAI/model/forest_reg.pkl' )

# Grid Search

In [ ]:
# we want to model performs better, in this case we tune the hyperparameters
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = [{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
              {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},]


In [ ]:
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                          return_train_score=True)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
# and see the best estimator
grid_search.best_estimator_


In [ ]:
# or best parameters
grid_search.best_params_

In [ ]:
# or see the how each combination has worked
cvres = grid_search.cv_results_

In [ ]:
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(round(np.sqrt(-mean_score),2), params)

Nhận xét: Trường hợp tốt nhất với dự báo ngẫu nhiên là 5.4.10^-6(g/m3)

# Analyze model

In [ ]:
# could look back to see how the weight of each input
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
sorted(zip(feature_importances, X.columns), reverse=True)


In [ ]:
# let see how grid search performs
# train set
grid_ytrain_p = grid_search.predict(X_train)
grid_mse = mean_squared_error(y_train, grid_ytrain_p)
grid_train_rmse = np.sqrt(grid_mse)
grid_train_rmse

In [ ]:
# let see how the prediction look like after hypertunning
# test set
grid_ytest_p = grid_search.predict(X_test)
grid_test_mse = mean_squared_error(y_test, grid_ytest_p)
grid_test_rmse = np.sqrt(grid_test_mse)
grid_test_rmse

In [ ]:
# still more to work with, but let bag the result for later comparison
add_stats(model='gridsearch',
         train_rmse=grid_train_rmse,
         test_rmse=grid_test_rmse)
results

In [ ]:
# let visualize it
std_ = grid_test_rmse
xindex = np.arange(0, len(y_test))
fig, ax = plt.subplots(figsize=(15,6))
ax.plot(y_test.to_list(), color='black', label='measured')
ax.plot(grid_ytest_p, color='red', alpha=0.8, label='predicted')
ax.fill_between(xindex, grid_ytest_p-std_,grid_ytest_p+std_,
                color='red', alpha=0.3, label='1signma' )
ax.fill_between(xindex, grid_ytest_p-2*std_,grid_ytest_p+2*std_,
                color='red', alpha=0.2, label = '2sigma')
ax.set_xlim(0,200)
ax.legend()

*Nhận* xét: Ta có thể thấy mô hình của Grid hoạt động khá tốt, biên độ rất nhỏ

# Scipy interval 95%

In [ ]:
# let use stats from scipy library
from scipy import stats

In [ ]:
# and look confidence of .95, or the area that a value will be inside the range with 95 chances of 100
confidence = 0.95

In [ ]:
squared_errors = (grid_ytest_p - y_test)**2

In [ ]:
np.sqrt(stats.t.interval(
    confidence,
    len(squared_errors)-1,
    loc=squared_errors.mean(),
    scale=stats.sem(squared_errors)))

- so we are pretty sure that standard deviation from grid search is from 1.3 to 4
- how confidience: 95 chances out of 100, this RMSE will be within this range

# Ensemble Methods

In [ ]:
# let look at a final approach to combine three regression we have so far using Voting method
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.tree import DecisionTreeRegressor


In [ ]:
# re-define an instance, all training in the previous sessions are gone
lin_reg  = LinearRegression()
tree_reg = DecisionTreeRegressor()
rnd_reg = RandomForestRegressor()


In [ ]:
# and make each model as an parameter for then ensemble (voting)
voting_reg = VotingRegressor(
    estimators=[('lin', lin_reg),
               ('rnd', rnd_reg),
               ('tree', tree_reg)
               ],
)


In [ ]:
# train model by the train set
voting_reg.fit(X_train, y_train)

In [ ]:
# check estimator (paramters)
voting_reg.estimators_


In [ ]:
# now we do predicting on the test set
y_entest_p = voting_reg.predict(X_test)

In [ ]:
mse = mean_squared_error(y_entest_p, y_test)
en_test = np.sqrt(mse)


In [ ]:
# we could run on train set
y_entrain_p = voting_reg.predict(X_train)
mse = mean_squared_error(y_entrain_p, y_train)
en_train = np.sqrt(mse)

In [ ]:
en_test, en_train

In [ ]:
# still in 21 for test set
add_stats(model='voting reg',
         train_rmse=en_train,
         test_rmse=en_test)
results

In [ ]:
# let visualize the data from Ensemble with test set
plt.figure(figsize=(15,5))
plt.plot(y_test.to_list())
plt.plot(y_entest_p, lw=1)
plt.xlim(200,300)

- so we can get a RMSE = 20 from several model, which is about 50% as the relative standard deviation
- this dataset is combined from several source, but not easy to get from a forecast product (in fact, I am struggling to get those), so we will try out a dataset with less feastures,

# DarkSky Dataset

- you can check out this API at DarkSky.net. After acquired by Apple, the future of this open API is unsure. The registration is closed as well. Alternatively, check out OpenWeatherMap.org


## Merge data

In [ ]:
# load data in
dk = pd.read_csv('data/darksky_hanoi_2018.csv', parse_dates=['time'], index_col=['time'])

In [ ]:
dk.columns

In [ ]:
# select few important columsn
cols = ['temperature', 'dewpoint', 'humidity', 'pressure', 'precipintensity','cloudcover', 'visibility', 'windspeed']

In [ ]:
dkt = dk[cols]

In [ ]:
# load PM2.5 data
pm = pd.read_csv('data/cleaned_pm25_Hanoi_PM2.5_2018_YTD.csv',
                parse_dates=['Date (LT)'],
                index_col=['Date (LT)'])


In [ ]:
pm.head()

In [ ]:
# check duplicated data if you want, wait, this is too much
dkt.duplicated().sum()


In [ ]:
dkt.info()

In [ ]:
# let sort index (datetime) first
dkt.sort_index(inplace=True)

- they are matched exact, but dropping them will need to fill in more data later, so it is okay to keep the closest values in by adjacent rows

In [ ]:
# merge data
df = pd.merge(pm, dkt, right_index=True, left_index=True, how='left')

In [ ]:
df.info()

In [ ]:
# quick check correlation
df.corr()['pm25'].sort_values()

In [ ]:
# seperate feature and label
X = df.drop('pm25', axis=1)
y = df['pm25'].copy()

In [ ]:
X_scaled = inpute_transfrom(data=X)

In [ ]:
type(X_scaled)

# Split train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y, test_size=0.33, random_state=2020)

In [ ]:
len(X_train), len(y_train)

In [ ]:
len(X_test), len(y_test)

# Train and validate

In [ ]:
# I will jump in and use voting (seem safer)
lin_reg  = LinearRegression()
tree_reg = DecisionTreeRegressor()
rnd_reg = RandomForestRegressor()
voting_reg = VotingRegressor(
    estimators=[('lin', lin_reg),
               ('rnd', rnd_reg),
               ('tree', tree_reg)
               ],
)

In [ ]:
voting_reg.fit(X_train, y_train)

In [ ]:
voting_reg.estimators_

## Trainset

In [ ]:
y_entrain_p = voting_reg.predict(X_train)


In [ ]:
mse = mean_squared_error(y_entrain_p, y_train)
train_std = np.sqrt(mse)
train_std

In [ ]:
y_train.iloc[0:100]

In [ ]:
from random import randint

In [ ]:
# time to invest some good graphs
def plot_results(label=None, prediction=None, std_=None, points=100, savefig=False):
    plt.style.use('default')
    plt.rcParams['font.size'] = 12
    start = randint(0, len(label)-points)
    end = start + points
    label_ = label.iloc[start:end]
    pred_ = prediction[start:end]
    xindex = np.arange(0,len(label_),1)

    plt.figure(figsize=(10,6))
    plt.plot(label_.to_list(), 'ko--',lw=0.5, label='measured')
    plt.plot(pred_, lw=1, color='#922B21', label='predicted')
    plt.fill_between(xindex, pred_- std_, pred_+std_,
                    color='#5499C7', alpha=0.6, label = '$\pm1 \sigma$')

    max_ = np.max([label_.max(), np.max(pred_)])
    plt.ylim(0, 1.1*max_)
    plt.ylabel('Concentration, $\mu g/m^3$')
    plt.title('Measured and predicted $PM_{2.5}$ using Ensemble regression',
              y=1.05, weight='bold')
    plt.xlabel('Hour, site: Hanoi, 2018')
    plt.legend(ncol=3)
    if savefig:
        plt.tight_layout()
        plt.savefig(f'img/en_reg_{start}.png', optimize=True)
    return None

In [ ]:
plot_results(label=y_train, prediction=y_entrain_p,
             std_=train_std, savefig=True)

## Testset

In [ ]:
y_entest_p = voting_reg.predict(X_test)

In [ ]:
mse = mean_squared_error(y_entest_p, y_test)
test_std = np.sqrt(mse)
mse, test_std


In [ ]:
add_stats(model='voting reg (Darksky)',
         train_rmse=train_std,
         test_rmse=test_std)
results

- not much worse, in fact, with less parameters and get a similar outcome, that is actually encouraging


In [ ]:
plot_results(label=y_test,
             prediction=y_entest_p, std_=test_std, savefig=True)

# RMSE

- again, this is Root Mean Squared Error. If we assumed the errors is random, then the distribution of error to the mean value shouls be in standard distribution (Gaussian Distribution). Then the RMSE is the Standard Deviation (SD). The ratio of SD to the mean value in percent is called Relative Standard Deviation.

In [ ]:
df = pd.DataFrame(data=results)

In [ ]:
df

In [ ]:
df2 = df.transpose()

In [ ]:
df2

In [ ]:
plt.style.use('seaborn-whitegrid')

In [ ]:
bw = 0.3
idx = np.arange(len(df2))
fig, ax = plt.subplots(figsize=(8,6))
ax.bar(idx-bw/2, df2['train_rmse'], bw, color='gray', label='train_set')
ax.bar(idx+bw/2, df2['test_rmse'], bw, color='navy', alpha=0.8, label='test_set')
ax.set_xticklabels(['','Linear', 'DecisionTree', 'RandomForest',
                    'Grid Search', 'Voting', 'Voting (DK*)'],
                  rotation=25)
ax.set_xlabel('Regression model, *DK: applied on DarkSky dataset; others: mixed-bag')
ax.set_title('Root Squared Mean Errors with $PM_{2.5}$ prediction\
             \n with meteorological parameters for Hanoi, 2018',
            y=1.05,
            weight='bold')
# labels = ax.get_xticklabels()
ax.set_ylabel('RMSE in $\mu g/m^3$')
ax.legend(frameon=True, ncol=2)
fig.tight_layout()
fig.savefig('img/2020Aug_rmse_raw.png')

In [ ]:
pm['pm25'].mean()

In [ ]:
df3 = df2*100/pm['pm25'].mean()

In [ ]:
df3

In [ ]:
bw = 0.3
idx = np.arange(len(df3))
fig, ax = plt.subplots(figsize=(8,6))
ax.bar(idx-bw/2, df3['train_rmse'], bw, color='gray', label='train_set')
ax.bar(idx+bw/2, df3['test_rmse'], bw, color='navy', alpha=0.8, label='test_set')
ax.set_xticklabels(['','Linear', 'DecisionTree', 'RandomForest',
                    'Grid Search', 'Voting', 'Voting (DK*)'],
                  rotation=25)
ax.set_xlabel('Regression model, *DK: applied on DarkSky dataset; others: mixed (MERRA-2, ISD)')
ax.set_title('$PM_{2.5}$ prediction\n with meteorological parameters for Hanoi, 2018',
             y=1.05,
            weight='bold')
# labels = ax.get_xticklabels()
ax.set_ylabel('Relative Standard Deviation, %')
ax.legend(frameon=True, ncol=2)
fig.tight_layout()
# fig.tight_layout(rect=(0.1,0.1,0.95, 0.95))
fig.savefig('img/2020Aug_rmse_rsd.png')